In [17]:
import numpy as np
from function import RBF_kernel,linear_kernel
from sklearn.metrics.pairwise import laplacian_kernel,rbf_kernel
import scipy
from scipy.stats import multivariate_normal
from scipy.stats._multivariate import _PSD
from scipy.stats._multivariate import multivariate_normal_gen
#np.random.seed(1234)
import time
import os
import psutil
import gc
import pandas as pd
from scipy.sparse import csr_matrix
# path = '/Users/lilingyu/E/PhD/Python/'
# path = '/storage3/yhhuang/users/lingyu/'
# path = '/data/users/lingyu/'
# path = '/home/lly/Python/'
#path = '/mnt/lingyu/nfs_share2/Python/'
path = '/mnt/jackywang/nfs_share2/spatialde'

In [7]:
b = [[0]*3] *3
b

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [3]:
row = np.array([0, 2, 1, 2, 2, 0])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
a = csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

In [8]:
l = [0,2]
a[l]

array([[1, 0, 6],
       [4, 5, 2]])

In [15]:
a

array([[1, 0, 6],
       [0, 0, 3],
       [4, 5, 2]])

In [16]:
b = a + np.identity(3)
rbf_kernel(a,b[[0,1]])

array([[7.16531311e-01, 2.55615332e-02],
       [1.31237287e-02, 7.16531311e-01],
       [3.05902321e-07, 1.67017008e-05]])

In [18]:
def RBF_kernel(X, l):
    X = np.array(X)
    Xsq = np.sum(np.square(X), 1)
    R2 = -2. * np.dot(X, X.T) + (Xsq[:, None] + Xsq[None, :])
    return np.exp(-R2 / (2 * l ** 2))

def linear_kernel(X):
    K = np.dot(X, X.T)
    return K / K.max()

In [19]:
class Kernel:

    def __init__(self,spatial,ss_loc,cov=None,dependency=None,d=5,kernel='laplacian',l=0.01):
        """
        number of superspots: M
        cov: full pre-determined covariance matrix
        dependency format: M-length list of lists, the i-th element indicates the superspots that the i-th superspot is dependent on
        ss_loc: M-length list of lists, the i-th element indicates the spots (ordinal) that the i-th superspots contain
        spatial: spatial coordinates of all spots
        l: hyperparameter for kernel
        """
        super().__init__()
        # if mean.ndim == 1:
        #     self.all_mean = np.array(mean)[:,np.newaxis]
        # else:
        #     self.all_mean = np.array(mean)       
        self.all_cov = cov
        self.dependency = dependency   # temporarily assuming dependency[i] > i 
        self.M = len(dependency)
        self.N = len(spatial)
        self.ss_loc = ss_loc
        self.spatial = spatial
        self.cond_mean = None
        self.l = l
        self.d = d
        self.A = []
        self.cond_cov = []
        self._initialize()
    
    def _initialize(self):
        self._init_ds_loc()
        self._init_ds_eig()
        self._init_base_cond_cov()        
        #self._init_cond_cov()

    def _init_ds_loc(self):
        self.ds_loc = []        
        for i,ds in enumerate(self.dependency):
           ind = []
           for ss in ds:
               ind += self.ss_loc[ss]
           self.ds_loc.append(ind)

            

    
    def _init_ds_eig(self):
        #C_m,C_m
        #print('de_eig')
        self.ds_eig = []
        for i,loc in enumerate(self.ds_loc):
            ds_cov = self.all_cov[np.ix_(loc,loc)]
            if len(ds_cov) == 0:
                self.ds_eig.append(())
                self.A.append(())
            else:
                s,u = scipy.linalg.eigh(ds_cov)
                #s_inv = _pinv_1d(s)
                self.ds_eig.append((s,u))
                self.A.append(self.all_cov[np.ix_(self.ss_loc[i],loc)] @ u)
    
    def _init_base_cond_cov(self):
        #print('base_cond')
        for i,loc in enumerate(self.ds_loc):
            if len(loc) == 0:
                self.cond_cov.append(self.all_cov[np.ix_(self.ss_loc[i],self.ss_loc[i])])
            else:
                self.cond_cov.append(self.all_cov[np.ix_(self.ss_loc[i],self.ss_loc[i])] - np.multiply(1/self.ds_eig[i][0],self.A[i])@self.A[i].T)

    def update_cond_cov(self,delta):
        self.cond_cov_eig = []
        for i in range(self.M):
            if len(self.dependency[i]) == 0:
                #self.cond_cov.append(self.kernel.base_cond_cov[i]+self.delta*np.eye(len(self.kernel.ss_loc[i])))
                s,u = scipy.linalg.eigh(self.cond_cov[i]+delta*np.eye(len(self.ss_loc[i])))
            else:
                s,u = scipy.linalg.eigh(self.cond_cov[i]+delta*np.eye(len(self.ss_loc[i]))+delta*np.multiply(1/((self.ds_eig[i][0]+delta)*self.ds_eig[i][0]),self.A[i])@self.A[i].T)
            self.cond_cov_eig.append((s,u))



class Gaussian:
    def __init__(self,kernel,mean=0,sigma_sq=1,delta=1) -> None:
        assert isinstance(kernel,Kernel)
        mean = np.array(mean)
        self.kernel = kernel
        # mean should be N*1
        if mean.ndim == 1:
            self.mean = np.array(mean)[:,np.newaxis]
        if mean.ndim == 0:
            self.mean = np.repeat(mean,kernel.N)[:,np.newaxis]
        self.sigma_sq = sigma_sq
        self.delta = delta
        self.cond_eig = []
        self.temp = []

    def update_cond_mean(self,Y):
        self.cond_dev = Y - self.mean
        dev = Y - self.mean
        for i in range(self.kernel.M):
            if len(self.kernel.dependency[i]) > 0:
                # print(self.delta)
                # print(np.multiply(1/(self.kernel.ds_eig[i][0]+self.delta),self.kernel.A[i]) @ self.kernel.ds_eig[i][1].T @ dev[self.kernel.ds_loc[i],:])
                self.cond_dev[self.kernel.ss_loc[i],:] = self.cond_dev[self.kernel.ss_loc[i],:] - np.multiply(1/(self.kernel.ds_eig[i][0]+self.delta),self.kernel.A[i]) @ self.kernel.ds_eig[i][1].T @ dev[self.kernel.ds_loc[i],:]


    def ll_sep(self,Y):
        """
        Y: observation of dimension N*G
        """
        N,G = Y.shape
        ll = np.log(2 * np.pi)*N + 2*np.log(self.sigma_sq)*N
        result = np.array([ll]*G)
        self.kernel.update_cond_cov(self.delta)
        self.update_cond_mean(Y)
        for i in range(self.kernel.M):
            det = np.prod(self.kernel.cond_cov_eig[i][0])
            result += np.log(det)
            # print(self.kernel.ss_loc[i])
            # print(self.kernel.A[i])
            temp = self.cond_dev[self.kernel.ss_loc[i],:].T @ self.kernel.cond_cov_eig[i][1]
            result += np.sum(np.multiply(1/self.kernel.cond_cov_eig[i][0],np.square(temp)),axis=1)/self.sigma_sq
        return result*-0.5

# Test1

In [7]:
N = 9000
spatial = []
dependency = []
for i in range(30):
    for j in range(30):
        spatial.append(np.random.multivariate_normal(0.5*np.array([j-1,1-i]),np.identity(2)*0.4,10))
        d = []
        for i_i in range(i-5,i+1):
            for j_j in range(j-4,j+5):
                if i_i > -1 and j_j > -1 and j_j < 30 and not(i_i == i and j_j > j):
                    d.append((i_i)*30+j_j)
        d.pop()
        dependency.append(d)

# N = 90
# spatial = []
# dependency = []
# for i in range(3):
#     for j in range(3):
#         spatial.append(np.random.multivariate_normal(0.5*np.array([j-1,1-i]),np.identity(2)*0.4,10))
#         d = []
#         for i_i in range(i-12,i+1):
#             for j_j in range(j-8,j+9):
#                 if i_i > -1 and j_j > -1 and j_j < 3 and not(i_i == i and j_j > j):
#                     d.append((i_i)*3+j_j)
#         d.pop()
#         dependency.append(d)
#         print(j-1,1-i,d)
spatial = np.vstack(spatial)
spatial.shape

(9000, 2)

In [5]:
mean = np.zeros(N)
cov = laplacian_kernel(spatial)
Y = np.random.multivariate_normal(mean,cov+np.identity(9000),2)
# cov = np.load('cov.npy')
# Y = np.load('Y.npy')
# spatial = np.load('spatial.npy')

In [7]:
pid = os.getpid()
p = psutil.Process(pid)
info = p.memory_full_info()
memory_1 = info.uss/1024

likelihood_4 = multivariate_normal.logpdf(Y, mean, cov+np.identity(9000))
likelihood_4

info = p.memory_full_info()
memory_2 = info.uss/1024

eps: 1.1422820943427486e-07


In [8]:
memory_2-memory_1

3348.0

In [9]:
memory_2

1760516.0

In [21]:
#ss_loc = [list(range(10*i, 10*(i+1))) for i in range(9)]
ss_loc = [list(range(10*i, 10*(i+1))) for i in range(900)]

pid = os.getpid()
p = psutil.Process(pid)
info = p.memory_full_info()
memory_1 = info.uss/1024

kernel = Kernel(spatial,ss_loc,cov=cov,dependency=dependency)

In [22]:
gaussian = Gaussian(kernel)

In [23]:
gaussian.ll_sep(Y.T)
info = p.memory_full_info()
memory_2 = info.uss/1024

In [24]:
gaussian.ll_sep(Y.T)

array([-13589.14626397, -13671.29626979])

In [25]:
likelihood_4

array([-13579.54258453, -13659.94757558])

In [31]:
memory_2

4444052.0

In [27]:
memory_2-memory_1

5212.0

In [29]:
import gc
del gaussian
del kernel
gc.collect()

0

In [32]:
pid = os.getpid()
p = psutil.Process(pid)
info = p.memory_full_info()
memory_1 = info.uss/1024
memory_1

4447788.0

# Test 2

In [20]:
def generate(m,i_scope,j_scope,s):
    N = 9000
    spatial = []
    dependency = []
    for i in range(m):
        for j in range(m):
            spatial.append(np.random.multivariate_normal(0.5*np.array([j-1,1-i]),np.identity(2)*0.7,s))
            d = []
            for i_i in range(i-i_scope,i+1):
                for j_j in range(j-j_scope,j+j_scope+1):
                    if i_i > -1 and j_j > -1 and j_j < m and not(i_i == i and j_j > j):
                        d.append((i_i)*m+j_j)
            d.pop()
            dependency.append(d)
    spatial = np.vstack(spatial)
    ss_loc = [list(range(s*i, s*(i+1))) for i in range(m**2)]
    return spatial,ss_loc,dependency
    

In [21]:
def pseudo_sim(mean,cov,N,g):
    ind = np.linspace(0,N,10,dtype=int)
    for i in range(9):
        new = np.random.multivariate_normal(mean[ind[i]:ind[i+1]],cov[ind[i]:ind[i+1],ind[i]:ind[i+1]]+np.identity(ind[i+1]-ind[i]),g)
        Y =  new if i==0 else np.hstack((Y,new))
    return Y

In [22]:
spatial,ss_loc,dependency = generate(30,3,3,5)
N = 5*30**2
mean = np.zeros(N)
cov = RBF_kernel(spatial,0.8)
Y = np.random.multivariate_normal(mean,cov+np.identity(N),5)

In [23]:
likelihood = multivariate_normal.logpdf(Y, mean, cov+np.identity(N))

In [24]:
likelihood

array([-6783.92715779, -6734.87975048, -6698.09411776, -6687.80778064,
       -6729.75249147])

In [25]:
kernel = Kernel(spatial,ss_loc,cov=cov,dependency=dependency)
gaussian = Gaussian(kernel)
likelihood_1 = gaussian.ll_sep(Y.T)

In [26]:
likelihood_1

array([-6827.17356776, -6764.96618717, -6732.45247582, -6744.98220111,
       -6779.65105874])

In [20]:
np.linalg.eigvalsh(cov)

array([-6.35247096e-13, -4.70420581e-13, -4.49749400e-13, ...,
        7.57788655e+01,  7.60803136e+01,  8.04527547e+01])

In [11]:
df = pd.DataFrame(columns=['N','i','j','M','G','Numpy time','Init time','Calc time','Acc'])

for m in range(20,80,5):
    for i_scope in range(3,11,2):
        for j_scope in range(1,9,2):
            for s in range(8,16,2):
                for g in range(2,52,30):
                    N = s*m**2
                    spatial,ss_loc,dependency = generate(m,i_scope,j_scope,s)


                    mean = np.zeros(N)
                    cov = laplacian_kernel(spatial)
                    Y = np.random.multivariate_normal(mean,cov+np.identity(N),g)

                    
                    start_time=time.time()
                    kernel = Kernel(spatial,ss_loc,cov=cov,dependency=dependency)
                    end_time=time.time()
                    time_0 = end_time - start_time                    

                    start_time=time.time()
                    likelihood = multivariate_normal.logpdf(Y, mean, cov+np.identity(N))
                    end_time=time.time()
                    time_1 = end_time - start_time

                    gaussian = Gaussian(kernel)
                    start_time=time.time()
                    likelihood_1 = gaussian.ll_sep(Y.T)
                    end_time=time.time()
                    time_2 = end_time - start_time                
                    
                    diff = likelihood_1-likelihood
                    acc = np.mean(diff/likelihood)
                    df2 = {'N':N,'i':i_scope,'j':j_scope,'M':m,'G':g,'Numpy time':time_1,'Init time':time_0,'Calc time':time_2,'Acc':acc}
                    df = df._append(df2, ignore_index = True)
                    df.to_csv('out.csv')


KeyboardInterrupt: 

In [5]:
# df = pd.read_csv('out_1.csv',index_col=0)
# for m in range(70,85,5):
#     for i_scope in range(3,11,2):
#         for j_scope in range(1,9,2):
#             for s in range(8,17,4):
#                 for g in range(2,53,30):
#                     N = s*m**2
#                     spatial,ss_loc,dependency = generate(m,i_scope,j_scope,s)


#                     mean = np.zeros(N)
#                     cov = laplacian_kernel(spatial)
#                     #Y = np.random.multivariate_normal(mean,cov+np.identity(N),g)
#                     Y = pseudo_sim(mean,cov,N,g)

                    
#                     start_time=time.time()
#                     kernel = Kernel(spatial,ss_loc,cov=cov,dependency=dependency)
#                     end_time=time.time()
#                     time_0 = end_time - start_time                    

#                     start_time=time.time()
#                     likelihood = multivariate_normal.logpdf(Y, mean, cov+np.identity(N))
#                     end_time=time.time()
#                     time_1 = end_time - start_time

#                     gaussian = Gaussian(kernel)
#                     start_time=time.time()
#                     likelihood_1 = gaussian.ll_sep(Y.T)
#                     end_time=time.time()
#                     time_2 = end_time - start_time                
                    
#                     diff = likelihood_1-likelihood
#                     acc = np.mean(diff/likelihood)
#                     df2 = {'N':N,'i':i_scope,'j':j_scope,'M':m,'G':g,'Numpy time':time_1,'Init time':time_0,'Calc time':time_2,'Acc':acc}
#                     df = df._append(df2, ignore_index = True)
#                     df.to_csv('out.csv')

[    0  4355  8711 13066 17422 21777 26133 30488 34844 39200]
de_eig
base_cond
[    0  4355  8711 13066 17422 21777 26133 30488 34844 39200]
de_eig
base_cond
[    0  6533 13066 19600 26133 32666 39200 45733 52266 58800]
de_eig
base_cond


In [5]:
df = pd.DataFrame(columns=['N','i','j','M','G','Numpy time','Init time','Calc time','Acc'])

for m in range(40,85,5):
    for i_scope in range(1,12,3):
        for j_scope in range(1,7,2):
            for s in range(8,17,4):
                for g in range(5,30,20):
                    N = s*m**2
                    spatial,ss_loc,dependency = generate(m,i_scope,j_scope,s)


                    mean = np.zeros(N)
                    cov = laplacian_kernel(spatial)
                    #Y = np.random.multivariate_normal(mean,cov+np.identity(N),g)
                    Y = pseudo_sim(mean,cov,N,g)

                    
                    start_time=time.time()
                    kernel = Kernel(spatial,ss_loc,cov=cov,dependency=dependency)
                    end_time=time.time()
                    time_0 = end_time - start_time     

                    start_time=time.time()
                    likelihood = multivariate_normal.logpdf(Y, mean, cov+np.identity(N))
                    end_time=time.time()
                    time_1 = end_time - start_time               

                    gaussian = Gaussian(kernel)
                    start_time=time.time()
                    likelihood_1 = gaussian.ll_sep(Y.T)
                    end_time=time.time()
                    time_2 = end_time - start_time                
                    
                    diff = likelihood_1-likelihood
                    acc = np.mean(diff/likelihood)
                    df2 = {'N':N,'i':i_scope,'j':j_scope,'M':m,'G':g,'Numpy time':time_1,'Init time':time_0,'Calc time':time_2,'Acc':acc}
                    df = df._append(df2, ignore_index = True)
                    df.to_csv('out.csv')